In [0]:
# Databricks: one-time installs for this notebook
%pip install nltk emoji scikit-learn

import nltk
nltk.download("vader_lexicon")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 30.7/40.5 kB 822.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 30.7/40.5 kB 822.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 364.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━ 51.2/57.7 kB 1.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.9 MB/s eta 0:00:01
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.6 MB/s eta 0:00:01
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/1.5 MB 1.7 MB/s eta 0:00:01
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 2.4 MB/s eta 0:00:01
   ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/1.5 MB 2.0 MB/s eta 0:00:01
   ━

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-a407f4c8-bc58-42c7-9d49-6f/nltk_data...


True

In [0]:
# ====== STORAGE CONFIG ======
ACCOUNT_KEY     = "U6rG7TnNjn8f5CUdcUbZKaZyvUHMuNvGAcVRlzczxCc3gRjvpiCP5Vi5Gqq4vqUARTZP3iFHS12v+AStV96cCw=="
storage_account = "goodreadsreviews60104310"
container       = "lakehouse"

base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net"
gold_path = f"{base_path}/gold"

# Configure Spark to use the key
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    ACCOUNT_KEY
)

print("Base path:", base_path)
print("Gold path:", gold_path)

# Quick check that we see the folders in 'gold'
display(dbutils.fs.ls(gold_path))


Base path: abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net
Gold path: abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold


path,name,size,modificationTime
abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold/curated_reviews/,curated_reviews/,0,1762277859000
abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold/features_v1/,features_v1/,0,1762534388000
abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold/features_v2/,features_v2/,0,1762961820000
abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold/features_v2_amaze/,features_v2_amaze/,0,1762548359000


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, length, size, split, udf
from pyspark.sql.types import StringType, DoubleType, StructType, StructField

import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download("vader_lexicon")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-a407f4c8-bc58-42c7-9d49-6f/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [0]:
# We start from the features_v1 dataset in the Gold layer
features_v1_path = f"{gold_path}/features_v1"

df = spark.read.format("delta").load(features_v1_path)

df.printSchema()
display(df.limit(5))


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- avg_rating_book: double (nullable = true)
 |-- num_reviews_book: long (nullable = true)



review_id,book_id,title,author_id,user_id,rating,review_text,review_length_words,avg_rating_book,num_reviews_book
0019b5c5d1d934aee34a1974331adb09,18335634,"Clockwork Princess (The Infernal Devices, #3)",150038,a85a90046a57add9210082aeb80acf56,5,wow what an ending I was sobbing x,8,4.649586125344896,2537
0039f4e8bc97cef177e1a3112cfcbf8d,23480844,Firsts,9816754,d8845bdde71a6093c5700fc0052ca278,3,"3.4 Que lindo libro. Tan diferente y tan comun a la vez. Me gusto demasiado. Los personajes, sus problemas. Como cada quien vive lo que le a tocado y cree en lo que desea. Es un libro demasiado bueno. Te ensena algunas lecciones importantes. Y sobre todo es un libro que te demuestra que la protagonista tambien puede equivocarse. Como todos nosotros.",62,3.8419540229885056,348
0128dbfb0b4869d9f85d5ed501d88fcc,247256,Forgotten Fire,144463,dc40a8cf7b68f805bf0b7eac883e6392,4,"I finished books two and three and finally got to feel what was going on throughout the whole novel. The inner peace that Vahan was looking for. This is an emotional piece that leads from the Armenians being taken from the homes they know to being slaves and being treated as though they do not exist. Vahan fights through all of the hardships he is faced until he is able to be free and not just free but is able to let go of everything he has faced; heartbreak, loss, starvation, molestation and even the feelings of love, adoration, admiration and caring for others. I think that if I were still able to teach the curriculum I had taught last year to my eighth graders this would be an awesome book to add in when we do the unit on Anne Frank so that students know that other people went through similar things like the Jewish people faced.",158,4.228571428571429,35
03935714bca04a32ccc16d6ed109d596,34150927,The Sound of the World By Heart,4462170,9f2aa698d867d91c108d7c96b4a7e903,4,"This well-written, emotionally resonant graphic novel is a keeper. The illustrations are stunning. The story telling is engaging. I highly recommend this. Thank you to Netgalley and Magnetic Press for the ARC.",32,3.772727272727273,44
04eb2b620033664780aa494f962b1e6f,15847315,Lady of Pleasure (School of Gallantry #3),1423331,a93164605da1931ce7592f23951ee688,3,"I am a big fan of Delilah Marvelle. Her novels are consistently good, with great story lines and great heat in the bedroom. That said, this book had one big problem. I would probably give this 3 1/2 stars if I could, just because it wasn't my favorite novel that she has done, and the heroine is really unlikeable. I was really shaking my head when the heroine rushed off to that house party, knowing the kind of scum that would be there. She really was clueless about her own safety and the possible outcome of her actions. I really felt sorry for the hero when he realizes what he'd done to her at the party. The heroine's anger towards the hero was a bit much, too, knowing how conflicted he felt before she tricked him into bed. And that is never a good way to win over your man, by tricking him into bed, by the way. She was lucky he loved her so much and was ultimately willing to work things out between them. She really infuriated me. On the other hand, I liked the hero a lot. He had a tragic backstory but still managed to become a decent person. Just like in lots of romance novels, I liked the hero way more than the heroine. I wish writers in this genre would work on making the heroine as compelling as the hero. When that rarity happens, I gladly bestow five stars!",245,4.411764705882353,17


In [0]:
train_df, val_df, test_df = df.randomSplit([0.7, 0.15, 0.15], seed=42)

train_n = train_df.count()
val_n   = val_df.count()
test_n  = test_df.count()
total_n = train_n + val_n + test_n

print(f"Train: {train_n}  ({train_n/total_n*100:.2f}%)")
print(f"Val:   {val_n}  ({val_n/total_n*100:.2f}%)")
print(f"Test:  {test_n}  ({test_n/total_n*100:.2f}%)")

simple_root = f"{gold_path}/features_v2/simple_features"

(train_df.write.format("delta").mode("overwrite")
    .save(f"{simple_root}/splits/train"))
(val_df.write.format("delta").mode("overwrite")
    .save(f"{simple_root}/splits/val"))
(test_df.write.format("delta").mode("overwrite")
    .save(f"{simple_root}/splits/test"))



Train: 10482219  (70.02%)
Val:   2243858  (14.99%)
Test:  2245294  (15.00%)


In [0]:
@F.udf(returnType=StringType())
def clean_text(text):
    if text is None:
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " url ", text)
    text = re.sub(r"\d+", " num ", text)
    text = re.sub(r"[^a-z\s]", " ", text)   # keep letters and spaces only
    text = re.sub(r"\s+", " ", text).strip()
    return text

train_clean = (
    train_df
    .withColumn("clean_review", clean_text(col("review_text")))
    .filter(length("clean_review") > 10)
    .withColumn("review_length_words", size(split(col("clean_review"), " ")))
    .withColumn("review_length_chars", length(col("clean_review")))
)

display(train_clean.select("review_text", "clean_review",
                           "review_length_words", "review_length_chars").limit(10))

train_clean_path = f"{simple_root}/clean_train"
train_clean.write.format("delta").mode("overwrite").save(train_clean_path)
print("Saved cleaned train to:", train_clean_path)


review_text,clean_review,review_length_words,review_length_chars
"Loved this book! The romance story was sweet, the plot was interesting, and I always appreciate Mitchell's attention to historical accuracy.",loved this book the romance story was sweet the plot was interesting and i always appreciate mitchell s attention to historical accuracy,22,136
"So I really enjoyed this story. I found the three narrator's, though unreliable and quite immoral, extremely fascinating. What makes people have an affair, or turn into drunks or even more, to murder someone. This book gave me insight into all of that and gave a thrill of a ride to boot. I enjoyed this one!",so i really enjoyed this story i found the three narrator s though unreliable and quite immoral extremely fascinating what makes people have an affair or turn into drunks or even more to murder someone this book gave me insight into all of that and gave a thrill of a ride to boot i enjoyed this one,57,299
"2.5 stars. Very ho-hum. Slightly predictable. Stayed up to finish it, but only because I wanted to just get it done with.",num num stars very ho hum slightly predictable stayed up to finish it but only because i wanted to just get it done with,24,120
"I hesitated to continue this book. The introduction held me back and prevented me from being engulfed by the book as books usually do to me. I held out because of the promise that this book held and I'm so glad I did. From the beginning this book is bleak, miserable, and full of despair. From the start you know this book is set amidst WWII but that is really never the focus of the book so much as a character in itself. Over the course of this novel no one is left unchanged and no one is left untouched by the war. Zusak's imagery may be hard to handle at first but by the end I realized there was simply no other way to tell this story. This may be the best book I've read in the past 5 years and even beyond that. After reading it I felt so raw. I can't stop thinking about the book, the characters, and what it meant to me. This book doesn't just stay on the pages- it seeps into your mind.",i hesitated to continue this book the introduction held me back and prevented me from being engulfed by the book as books usually do to me i held out because of the promise that this book held and i m so glad i did from the beginning this book is bleak miserable and full of despair from the start you know this book is set amidst wwii but that is really never the focus of the book so much as a character in itself over the course of this novel no one is left unchanged and no one is left untouched by the war zusak s imagery may be hard to handle at first but by the end i realized there was simply no other way to tell this story this may be the best book i ve read in the past num years and even beyond that after reading it i felt so raw i can t stop thinking about the book the characters and what it meant to me this book doesn t just stay on the pages it seeps into your mind,185,883
"When young Liesel is left with her new foster parents after her brother dies, she develops a passion for words and books and through her friendships with her foster father, her neighborhood friend Rudy, and a Jew her family is keeping hidden Max, she learns many different things about words and the importance of them. The Book Thief is written very creatively. I can see why it's a Printz honor book. But honestly, it didn't really stand out to me. It didn't always grab my attention and sometimes it even put me to sleep. It's a very important story though. It's good but not my favorite. Warning: language, death and violence.",when young liesel is left with her new foster parents after her brother dies she develops a passion for words and books and through her friendships with her foster father her neighborhood friend rudy and a jew her family is keeping hidden max she learns many different things about words and the importance of them the book thief is written very creativ

Saved cleaned train to: abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold/features_v2/simple_features/clean_train


In [0]:
train_clean = spark.read.format("delta").load(train_clean_path)

sia = SentimentIntensityAnalyzer()

def vader_scores(text):
    if not text:
        return (0.0, 0.0, 0.0, 0.0)
    s = sia.polarity_scores(text)
    return (
        float(s["pos"]),
        float(s["neu"]),
        float(s["neg"]),
        float(s["compound"])
    )

sent_schema = StructType([
    StructField("pos", DoubleType()),
    StructField("neu", DoubleType()),
    StructField("neg", DoubleType()),
    StructField("compound", DoubleType()),
])

sent_udf = udf(vader_scores, sent_schema)

train_sent = (
    train_clean
    .withColumn("sent", sent_udf("clean_review"))
    .withColumn("sentiment_pos", col("sent.pos"))
    .withColumn("sentiment_neu", col("sent.neu"))
    .withColumn("sentiment_neg", col("sent.neg"))
    .withColumn("sentiment_compound", col("sent.compound"))
    .drop("sent")
)

display(train_sent.select("clean_review",
                          "sentiment_pos", "sentiment_neu",
                          "sentiment_neg", "sentiment_compound").limit(10))

train_sent_path = f"{simple_root}/sentiment_train"
train_sent.write.format("delta").mode("overwrite").save(train_sent_path)
print("Saved sentiment train to:", train_sent_path)




clean_review,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound
sole otero es una increible dibujante y tiene una gran capacidad para graficar cosas que nos suceden a todos totalmente recomendable para los amantes de las novelas graficas,0.0,1.0,0.0,0.0
this book was even better than the first one for me i read it in less than two days and loved every minute of it i loved the plot to this book nora and patch aren t hanging out as much anymore mostly because of unfortunate events and a little of nora s doing yet she still wants to see him all the time in this book we find out more about how the archangels and the nephilim are created or made or how they interact and stuff like that we meet more people and a lot of dirty secrets are brought out we see a lot more of marcie in this one can t say i like marcie but this book may make you think differently about her oh and she seems to be spending a lot of time with patch we also meet more characters i have to say a character i thought was bad was actually not important at all and one i thought was good was definitely not i really have to say that one of the bad guys i wasn t expecting at all at all i even liked him i couldn t believe the ending though she s so mean for ending it there i m really anticipating the next book i would definitely recommend,0.132,0.801,0.068,0.9083
excellent couldn t put it down,0.481,0.519,0.0,0.5719
a fast paced story a good beach read lulu takes a vacation from her high tech lawyer job and finds love that she wasn t looking for a heartfelt love story,0.402,0.598,0.0,0.9413
am i a sf philistine clarke s book bogged right the hell down for me i put it down for weeks and only picked it back up cause i was bloody minded i remember reading other of his books probably childhood s end num and rendezvous with rama and enjoying them when i was a teen but i found the book had most of the sins of early sf a cypher for a main character completely cut off from anything that would ground the book in some sort of believable world interesting ideas but it just could not hold me even for such a short book such a very long short book okay so this is me talking completely out of my ass but i do wonder if clarke being gay at a time when it was not okay to be gay is a part of what makes this book so terrible for me i m really glad i went back to the book because one of the later sections is from the point of view of alvin s best friend in the novel hilvar the other half of the human beings who are telepathic and much more in touch with their feelings and the world around them if the book had been from hilvar s point of view i probably could have gotten into the story but it might have been much more dangerous for clarke because at least from the vantage point of today i think hilvar is in love with alvin this might be me reading modern sensibilities into a story re written in the num s yet alvin is simply not interested in the female love interest in the book and does want to hang around with his friend num s code for gayness perhaps this is clarke being brilliant he s written a novel about human beings cut off from their human quality with a main character who can t admit his own desires yet i found the book so dry and alienating that i nearly didn t make my way through it i think it is a flawed book in the sense that like many other reviewers i think that once alvin gets outside the city the book flags seriously and only picks up again right at the very end where there is a burst of not action but explanation that such a small book can have such a flabby middle is quite the accomplishment still it s a interesting book both for the author s huge ego and conflicted sexuality as well as some of its ideas alvin s learned humility was quite amusing one of clarke s constant motifs is about what comes beyond mankind yet in this novel hand waving about transcendence seems like yet another psychological escape for discomfort over the body and what it really wants this is

Saved sentiment train to: abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold/features_v2/simple_features/sentiment_train


In [0]:
train_sent = spark.read.format("delta").load(train_sent_path)

def avg_word_length(text):
    if not text:
        return 0.0
    words = text.split()
    if not words:
        return 0.0
    return float(sum(len(w) for w in words) / len(words))

def lexical_diversity(text):
    if not text:
        return 0.0
    words = text.split()
    if not words:
        return 0.0
    return float(len(set(words)) / len(words))

avg_len_udf = udf(avg_word_length, DoubleType())
lex_div_udf = udf(lexical_diversity, DoubleType())

train_features = (
    train_sent
    .withColumn("avg_word_length", avg_len_udf("clean_review"))
    .withColumn("lexical_diversity", lex_div_udf("clean_review"))
)

display(train_features.select(
    "clean_review",
    "review_length_words",
    "review_length_chars",
    "sentiment_pos",
    "sentiment_compound",
    "avg_word_length",
    "lexical_diversity"
).limit(10))



clean_review,review_length_words,review_length_chars,sentiment_pos,sentiment_compound,avg_word_length,lexical_diversity
sole otero es una increible dibujante y tiene una gran capacidad para graficar cosas que nos suceden a todos totalmente recomendable para los amantes de las novelas graficas,28,173,0.0,0.0,5.214285714285714,0.9285714285714286
this book was even better than the first one for me i read it in less than two days and loved every minute of it i loved the plot to this book nora and patch aren t hanging out as much anymore mostly because of unfortunate events and a little of nora s doing yet she still wants to see him all the time in this book we find out more about how the archangels and the nephilim are created or made or how they interact and stuff like that we meet more people and a lot of dirty secrets are brought out we see a lot more of marcie in this one can t say i like marcie but this book may make you think differently about her oh and she seems to be spending a lot of time with patch we also meet more characters i have to say a character i thought was bad was actually not important at all and one i thought was good was definitely not i really have to say that one of the bad guys i wasn t expecting at all at all i even liked him i couldn t believe the ending though she s so mean for ending it there i m really anticipating the next book i would definitely recommend,221,1061,0.132,0.9083,3.8054298642533935,0.5248868778280543
excellent couldn t put it down,6,30,0.481,0.5719,4.166666666666667,1.0
a fast paced story a good beach read lulu takes a vacation from her high tech lawyer job and finds love that she wasn t looking for a heartfelt love story,31,154,0.402,0.9413,4.0,0.8387096774193549
am i a sf philistine clarke s book bogged right the hell down for me i put it down for weeks and only picked it back up cause i was bloody minded i remember reading other of his books probably childhood s end num and rendezvous with rama and enjoying them when i was a teen but i found the book had most of the sins of early sf a cypher for a main character completely cut off from anything that would ground the book in some sort of believable world interesting ideas but it just could not hold me even for such a short book such a very long short book okay so this is me talking completely out of my ass but i do wonder if clarke being gay at a time when it was not okay to be gay is a part of what makes this book so terrible for me i m really glad i went back to the book because one of the later sections is from the point of view of alvin s best friend in the novel hilvar the other half of the human beings who are telepathic and much more in touch with their feelings and the world around them if the book had been from hilvar s point of view i probably could have gotten into the story but it might have been much more dangerous for clarke because at least from the vantage point of today i think hilvar is in love with alvin this might be me reading modern sensibilities into a story re written in the num s yet alvin is simply not interested in the female love interest in the book and does want to hang around with his friend num s code for gayness perhaps this is clarke being brilliant he s written a novel about human beings cut off from their human quality with a main character who can t admit his own desires yet i found the book so dry and alienating that i nearly didn t make my way through it i think it is a flawed book in the sense that like many other reviewers i think that once alvin gets outside the city the book flags seriously and only picks up again right at the very end where there is a burst of not action but explanation that such a small book can have such a flabby middle is quite the accomplishment still it s a interesting book both for the author s huge ego and conflicted sexuality as well as some of its ideas alvin s learned humility was quite amusing one of clarke s constant motifs is about what comes beyond mankind y

In [0]:
final_train_path = f"{simple_root}/final_train"

train_features.write.format("delta").mode("overwrite").save(final_train_path)

print("Final feature dataset saved to:", final_train_path)

final_df = spark.read.format("delta").load(final_train_path)
final_df.printSchema()
display(final_df.limit(10))


Final feature dataset saved to: abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold/features_v2/simple_features/final_train
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- avg_rating_book: double (nullable = true)
 |-- num_reviews_book: long (nullable = true)
 |-- clean_review: string (nullable = true)
 |-- review_length_chars: integer (nullable = true)
 |-- sentiment_pos: double (nullable = true)
 |-- sentiment_neu: double (nullable = true)
 |-- sentiment_neg: double (nullable = true)
 |-- sentiment_compound: double (nullable = true)
 |-- avg_word_length: double (nullable = true)
 |-- lexical_diversity: double (nullable = true)



review_id,book_id,title,author_id,user_id,rating,review_text,review_length_words,avg_rating_book,num_reviews_book,clean_review,review_length_chars,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound,avg_word_length,lexical_diversity
00014ffcc6c6072738d3ba38088a3a96,25823468,La pelusa de los días,4582173,58eb42061b38d5e66452435062c56e40,5,Sole Otero es una increible dibujante y tiene una gran capacidad para graficar cosas que nos suceden a todos. Totalmente recomendable para los amantes de las novelas graficas,28,5.0,1,sole otero es una increible dibujante y tiene una gran capacidad para graficar cosas que nos suceden a todos totalmente recomendable para los amantes de las novelas graficas,173,0.0,1.0,0.0,0.0,5.214285714285714,0.9285714285714286
0001fd6f841bbeaf4678908a648cc2ae,9482584,"Crescendo (Hush, Hush, #2)",2876763,bca072e7a90fb8a0ca595e12a17d1bba,5,"This book was even better than the first one for me. I read it in less than two days and loved every minute of it. I loved the plot to this book. Nora and Patch aren't hanging out as much anymore, mostly because of unfortunate events and a little of Nora's doing. Yet, she still wants to see him all the time. In this book we find out more about how the Archangels and the Nephilim are created or made or how they interact and stuff like that. We meet more people and a lot of dirty secrets are brought out. We see a lot more of Marcie in this one. Can't say I like Marcie but this book may make you think differently about her. Oh and she seems to be spending a lot of time with Patch. We also meet more characters. I have to say a character I thought was bad was actually not important at all and one I thought was good was definitely not. I really have to say that one of the bad guys I wasn't expecting at all. AT ALL. I even liked him :/ I couldn't believe the ending though!! She's so mean for ending it there. I'm really anticipating the next book! I would definitely recommend :)",221,3.76,25,this book was even better than the first one for me i read it in less than two days and loved every minute of it i loved the plot to this book nora and patch aren t hanging out as much anymore mostly because of unfortunate events and a little of nora s doing yet she still wants to see him all the time in this book we find out more about how the archangels and the nephilim are created or made or how they interact and stuff like that we meet more people and a lot of dirty secrets are brought out we see a lot more of marcie in this one can t say i like marcie but this book may make you think differently about her oh and she seems to be spending a lot of time with patch we also meet more characters i have to say a character i thought was bad was actually not important at all and one i thought was good was definitely not i really have to say that one of the bad guys i wasn t expecting at all at all i even liked him i couldn t believe the ending though she s so mean for ending it there i m really anticipating the next book i would definitely recommend,1061,0.132,0.801,0.068,0.9083,3.8054298642533935,0.5248868778280543
00029bd67fd37a8e64bf3ac3b5821d64,7624086,"Hellhound on His Trail: The Stalking of Martin Luther King, Jr. and the International Hunt for His Assassin",54521,baf61428e36c7aa50b62a4db76a2496f,4,Excellent; couldn't put it down.,6,4.477064220183486,109,excellent couldn t put it down,30,0.481,0.519,0.0,0.5719,4.166666666666667,1.0
0002d1b1ed116317ad024aa281e83bf5,16669777,Dilemma at Bayberry Cove,326860,153fd3f47047eba84223371c6bc71ad5,3,"A fast paced story. A good beach read. LuLu takes a vacation from her high tech lawyer job, and finds love that she wasn't looking for. A heartfelt love story",31,3.0,1,a fast paced story a good beach read lulu takes a vacation from her high tech lawyer job and finds love that she wasn t looking for a heartfelt love story,154,0.402,0.598,0.0,0.9413,4.0,0.8387096774193549
00031890a1f6b520822bf2a2a91691b6,149088,The City and the Stars,7779,6865598a640

In [0]:
# We already have: train_features from the previous cell
# (sentiment + length + lexical features +
#  plus all the original metadata columns from features_v1)

final_train = train_features.select(
    "review_id",
    "book_id",
    "rating",
    "clean_review",
    "review_length_words",
    "review_length_chars",
    "sentiment_pos",
    "sentiment_neu",
    "sentiment_neg",
    "sentiment_compound",
    "avg_word_length",
    "lexical_diversity"
)

display(final_train.limit(10))
print("Rows in final_train:", final_train.count())


review_id,book_id,rating,clean_review,review_length_words,review_length_chars,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound,avg_word_length,lexical_diversity
00014ffcc6c6072738d3ba38088a3a96,25823468,5,sole otero es una increible dibujante y tiene una gran capacidad para graficar cosas que nos suceden a todos totalmente recomendable para los amantes de las novelas graficas,28,173,0.0,1.0,0.0,0.0,5.214285714285714,0.9285714285714286
0001fd6f841bbeaf4678908a648cc2ae,9482584,5,this book was even better than the first one for me i read it in less than two days and loved every minute of it i loved the plot to this book nora and patch aren t hanging out as much anymore mostly because of unfortunate events and a little of nora s doing yet she still wants to see him all the time in this book we find out more about how the archangels and the nephilim are created or made or how they interact and stuff like that we meet more people and a lot of dirty secrets are brought out we see a lot more of marcie in this one can t say i like marcie but this book may make you think differently about her oh and she seems to be spending a lot of time with patch we also meet more characters i have to say a character i thought was bad was actually not important at all and one i thought was good was definitely not i really have to say that one of the bad guys i wasn t expecting at all at all i even liked him i couldn t believe the ending though she s so mean for ending it there i m really anticipating the next book i would definitely recommend,221,1061,0.132,0.801,0.068,0.9083,3.8054298642533935,0.5248868778280543
00029bd67fd37a8e64bf3ac3b5821d64,7624086,4,excellent couldn t put it down,6,30,0.481,0.519,0.0,0.5719,4.166666666666667,1.0
0002d1b1ed116317ad024aa281e83bf5,16669777,3,a fast paced story a good beach read lulu takes a vacation from her high tech lawyer job and finds love that she wasn t looking for a heartfelt love story,31,154,0.402,0.598,0.0,0.9413,4.0,0.8387096774193549
00031890a1f6b520822bf2a2a91691b6,149088,3,am i a sf philistine clarke s book bogged right the hell down for me i put it down for weeks and only picked it back up cause i was bloody minded i remember reading other of his books probably childhood s end num and rendezvous with rama and enjoying them when i was a teen but i found the book had most of the sins of early sf a cypher for a main character completely cut off from anything that would ground the book in some sort of believable world interesting ideas but it just could not hold me even for such a short book such a very long short book okay so this is me talking completely out of my ass but i do wonder if clarke being gay at a time when it was not okay to be gay is a part of what makes this book so terrible for me i m really glad i went back to the book because one of the later sections is from the point of view of alvin s best friend in the novel hilvar the other half of the human beings who are telepathic and much more in touch with their feelings and the world around them if the book had been from hilvar s point of view i probably could have gotten into the story but it might have been much more dangerous for clarke because at least from the vantage point of today i think hilvar is in love with alvin this might be me reading modern sensibilities into a story re written in the num s yet alvin is simply not interested in the female love interest in the book and does want to hang around with his friend num s code for gayness perhaps this is clarke being brilliant he s written a novel about human beings cut off from their human quality with a main character who can t admit his own desires yet i found the book so dry and alienating that i nearly didn t make my way through it i think it is a flawed book in the sense that like many other reviewers i think that once alvin gets outside the city the book flags seriously and only picks up again right at the very end where there is a burst of not action but explanatio

Rows in final_train: 10410631


In [0]:
# Path for the final training dataset required by the lab
final_train_path = f"{gold_path}/features_v2/train_simple_features"

final_train.write \
    .format("delta") \
    .mode("overwrite") \
    .save(final_train_path)

print("Final training features saved to:", final_train_path)


Final training features saved to: abfss://lakehouse@goodreadsreviews60104310.dfs.core.windows.net/gold/features_v2/train_simple_features
